In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import time
import json
from operator import itemgetter
from tqdm.notebook import tqdm
from collections import OrderedDict

In [ ]:
#medicine data
medicine = pd.read_csv("/content/drive/MyDrive/Internship (Retracing Project)/Data/Progress 1 (Gath &Cl)/clean_medicines_091121.csv")
medicinePresc = pd.read_csv("/content/drive/MyDrive/Internship (Retracing Project)/Data/Progress 1 (Gath &Cl)/clean_medicine_prescription_091121.csv")
medicine_dose = pd.read_csv("/content/drive/MyDrive/Internship (Retracing Project)/Data/Progress 4 (Collaborative Filtering)/meds_with_dose.csv")

#patient & precription data
patients = pd.read_csv("/content/drive/MyDrive/Internship (Retracing Project)/Data/Progress 2 (Preprocessing)/processed_patient.csv")
fullPresc = pd.read_csv("/content/drive/MyDrive/Internship (Retracing Project)/Data/Progress 3 (Data Splitting)/80 Percent Used/prescription_reduced.csv")
allPivot = pd.read_csv("/content/drive/MyDrive/Internship (Retracing Project)/Data/Progress 1 (Gath &Cl)/clean_prescription_pivot_report_051121.csv")

In [ ]:
#Drug Map data
with open("/content/drive/MyDrive/Internship (Retracing Project)/Data/Progress 1 (Gath &Cl)/ICD-10 Drug Mapping/clean_drug_mapping.json",'r') as f:
  drugMap = json.load(f)

drugMap = OrderedDict(drugMap)

In [ ]:
#Drug Map data
with open("/content/drive/MyDrive/Internship (Retracing Project)/Data/json/Medicine Mapping Reference/drugMappingCleaned.json",'r') as f:
  drugMap = json.load(f)

drugMap = OrderedDict(drugMap)

In [ ]:
def similarityICD10(testPresc,trainPresc):
  testPresc = json.loads(testPresc)
  trainPresc = json.loads(trainPresc)
  count = 0
  for i in range(4):
    if len(trainPresc) < 5:
      trainPresc.append('')
    if len(testPresc) < 5:
      testPresc.append('')
  for i, value in enumerate(testPresc):
    if value in trainPresc:
      count+= 1/((abs(trainPresc.index(value) - testPresc.index(value)))+1)
  similarity = count/len(testPresc)
  return similarity

In [ ]:
def similarityPatientData(testPatients, trainPatients):
  ageCategory = ['Child', 'Adolescence', 'Adult', 'Senior Adult']
  BMICategory = ['Underweight','Normal','Overweight']
  ageTest = ageCategory.index(testPatients['patientAgeCategory'].item())
  ageTrain = ageCategory.index(trainPatients['patientAgeCategory'].item())
  BMITest = BMICategory.index(testPatients['patientBMICategory'].item())
  BMITrain = BMICategory.index(trainPatients['patientBMICategory'].item())
  ageDistance = abs(ageTest - ageTrain)
  BMIDistance = abs(BMITest - BMITrain)
  finalDistance = ageDistance + BMIDistance
  return finalDistance

In [ ]:
def filterICD10(testI,train, K):
  dictTemp = dict()
  for index, row in train.iterrows():
    sim = similarityICD10(testI['differentialDiagnosis'], row['differentialDiagnosis'])
    dictTemp[row['prescriptionNumber']] = {'Similarity': sim, 'id':row['patientId']}
  topK = dict(sorted(dictTemp.items(), key=lambda item: item[1]['Similarity'],reverse=True)[:K])
  #print(topK)
  return topK

In [ ]:
def filterPatientData(testI, train, testPatients, trainPatients, K):
  #print(testPatients['id'])
  #print(testI['patientId'])
  testRow = testPatients.loc[testPatients['id'] == testI['patientId']]
  for i in train:
    trainRow = trainPatients.loc[trainPatients['id'] == train[i]['id']]
    train[i]['patientSimilarity'] = similarityPatientData(testRow,trainRow)
  sortedDict = dict(sorted(train.items(), key=lambda item: (item[1]['Similarity'], item[1]['patientSimilarity']),reverse=True)[:K])
  return sortedDict

In [ ]:
def getMedicineAboveThres(topK, allPivot):
  topPresc = topK.keys()
  allMedList = dict()
  prescResult = allPivot[allPivot['prescriptionNumber'].isin(topPresc)]
  for index, row in prescResult.iterrows():
    tempString = 'medicineBrand'
    for i in range(5):
        stringCounter = str(i+1)
        meds = tempString+stringCounter
        if (not pd.isnull(row[meds])):
            if(row[meds] not in allMedList):
                allMedList[row[meds]] = 1

            else:
                allMedList[row[meds]] +=1
  return dict(sorted(allMedList.items(), key=itemgetter(1), reverse=True)[:10])

In [ ]:
def findTopMed(drugMap):
  topMed = dict()
  icd10 = drugMap.keys()
  medicine = drugMap.values()
  for i in icd10:
    for med in medicine:
      #print(med)
      listCount = list(med.values())
      represMed = list(med.keys())
      #print(represMed)
      #print(listCount)
      #represMed = next(iter(med))
      floatCount = [float(item) for item in listCount]
      #print(floatCount)
      #for num in floatCount:
        #sumit = sum(floatCount[num])
      represNum = (floatCount[0]) / sum(floatCount)
      #topMed = {represMed[0]: represNum}
      topMed[represMed[0]] = represNum
  #print(topMed)    
  #topMed = sorted(topMed.items(), key=itemgetter(1), reverse=True)
  topMed = {k: v for k, v in sorted(topMed.items(), key=lambda v: v[1], reverse=True)}
  return topMed

In [ ]:
def getBest10Med(topMed):
  best10Med = dict()
  threshold = 0.04
  for med in topMed:
    score = topMed[med]
    if score >= threshold:
          if med not in best10Med:
              best10Med[med] = 1
          else:
              best10Med[med] += 1
    else:
          pass
  #print(best10Med)
  #best10Med = sorted(top10Med.items(), key=itemgetter(1), reverse=True)[:10]
  best10Med = {k: v for k, v in sorted(best10Med.items(), key=lambda v: v[1], reverse=True)[:10]}
  return best10Med

In [ ]:
def getMedicineBelowThres(drugMap):
  topMed = findTopMed(drugMap)
  #print(topMed)
  best10Med = getBest10Med(topMed)
  return best10Med

In [ ]:
medListBelowThres = getMedicineBelowThres(drugMap)
print(medListBelowThres)

{'VITAMIN C TAB 250 MG Tablet': 1, 'ELKANA Syrup Botol 60ML': 1, 'ASAM MEFENAMAT 500 MG Tablet': 1, 'EXTRALAC LAKTULOSA SYRUP 10 G/15 ML, BOTOL 120 ML': 1, 'KETOCONAZOLE CREAM 0.02 Tube': 1, 'CETIRIZINE 10 MG Tablet': 1, 'PARACETAMOL 500 MG Tablet': 1, 'BETAHISTIN 6 MG Tablet': 1, 'GENTAMICIN 0.003 Salep Mata Tube': 1, 'PARACETAMOL 600 MG Tablet': 1}


In [ ]:
def getMedicine(topK, allPivot, medListBelowThres):
  tempString = "Similarity"
  simScore =  [sub[tempString] for sub in topK.values() if tempString in sub.keys()] #dia ngambil semua nilai sim score trs dijadiin list
  for score in simScore: #score is the index
      if score >= 0.35:
          medList =  getMedicineAboveThres(topK, allPivot)
      else:
          medList = medListBelowThres
  #print(medList)
  return medList

In [ ]:
def getGTMedicine(testRow):
  medList = dict()
  tempString = 'medicineBrand'
  for i in range(5):
      stringCounter = str(i + 1)
      meds = tempString + stringCounter
      if (not pd.isnull(testRow[meds].item())):
        if (testRow[meds].item() not in medList or testRow[meds].item() is not np.nan):
          medList[testRow[meds].item()] = 1
  return medList

In [ ]:
def getMedicineDosage(medList):
  doseDict = dict()
  medError = ["OBH 200 IKAP sir Komb btl 200 ml", "Nystatin NOVE susp100.000 IU/ml btl 15 ml"]
  medBrand = [*medList]
  #print(medBrand)
  for i in medError:
    if i in medBrand: medBrand.remove(i)
  if len(medBrand)==0:
    pass
  else:
    for brand in medBrand:
      if brand in medicine_dose['brand']:
        doseDict[brand] = {'frequency': medicine_dose.loc[medicine_dose['brand'] == brand, 'frequency'].values[0],
                           'frequencyDd': medicine_dose.loc[medicine_dose['brand'] == brand, 'frequencyDd'].values[0],
                           'timing': medicine_dose.loc[medicine_dose['brand'] == brand, 'timing'].values[0],
                           'duration': medicine_dose.loc[medicine_dose['brand'] == brand, 'duration'].values[0],
                           'amount': medicine_dose.loc[medicine_dose['brand'] == brand, 'amount'].values[0]}
      else:
        pass
  return doseDict

In [ ]:
def prepro(fullPresc, patients, allPivot):
  patientIds = set(fullPresc['patientId'])
  prescIds = set(fullPresc['id'])
  realPatients = patients[patients['id'].isin(patientIds)]
  realPivot = allPivot[allPivot['id'].isin(prescIds)]
  realPresc = fullPresc[fullPresc['id'].isin(realPivot['id'])]
  return realPresc, realPatients, realPivot

In [ ]:
def predict(testData,trainData,testPivot, trainPivot,testPatient,trainPatient,kICD10, kPatient, medListBelowThres):
  start = time.perf_counter()
  filteredICD10 = filterICD10(testData, trainData, kICD10)
  filteredPatientData = filterPatientData(testData, filteredICD10, testPatient, trainPatient, kPatient)
  #print(filteredPatientData)
  #medListBelowThres = getMedicineBelowThres(drugMap)
  result = getMedicine(filteredPatientData, trainPivot, medListBelowThres)
  testPivot1 = testPivot.loc[testPivot['prescriptionNumber'] == testData['prescriptionNumber']]
  groundTruth = getGTMedicine(testPivot1)
  #print(f'Predicted: {result}')
  medWithDosage = getMedicineDosage(result)
  #print(f'Predicted with Dose: {medWithDosage}')
  # print(f'Ground Truth: {groundTruth}')
  recall = reCall(result, groundTruth)
  precision = Precision(result, groundTruth)
  f1score = f1Score(recall, precision)
  # print(f'Recall = {recall}')
  end = time.perf_counter()
  runtime = end - start
  return result, recall, precision, f1score, runtime



---

**Preprocess Data**

In [ ]:
#preprocess Data
realPresc, realPatients, realPivot = prepro(fullPresc, patients, allPivot)

#Split Data
#trainPresc, testPresc = train_test_split(realPresc, test_size=0.2, random_state=0)
#trainPatients, testPatients = train_test_split(realPatients, test_size=0.2, random_state=0)
#trainPivot, testPivot = train_test_split(realPivot, test_size=0.2, random_state=0)



---
**Evaluation**


In [ ]:
def reCall(prediction, groundTruth):
    count = 0
    for i in groundTruth:
        if i in prediction:
            count += 1
    if len(groundTruth) > 0 :
        recall = float(count / len(groundTruth))
        return recall
    else:
        return 0

In [ ]:
def Precision(prediction, groundTruth):
  count = 0
  for i in groundTruth:
    if i in prediction:
        count += 1
  if len(prediction) > 0:
    precision = float(count / len(prediction))
    return precision
  else:
    return 0

In [ ]:
def f1Score(recall, precision):
  if (float(precision + recall) > 0):
    f1score = float(float(2) * (float(precision * recall)) / (float(precision + recall)))
    return f1score 
  else:
    return 0 

In [ ]:
def splitKFold(data, k):
  dataCollection = dict()
  for i in range(1, k):
    test = data.iloc[(int(len(data)/k)*i):(int(len(data)/k)*(i+1))]
    train = data[~data['prescriptionNumber'].isin(test['prescriptionNumber'])]
    dataCollection[i] = dict()
    dataCollection[i]['Test'] = test
    dataCollection[i]['Train'] = train
  return dataCollection

In [ ]:
realPatients.shape

(6342, 45)

In [ ]:
def validate(data, k):
  medListBelowThres = getMedicineBelowThres(drugMap)
  allScoreSum = dict()
  for i in tqdm(data):
    print(f'Start of {(i)}-st/th Fold Validation')
    print('=================================')
    trainSet = data[i]['Train']
    testSet = data[i]['Test']
    # print(testSet.info)
    trainData, trainPatients, trainPivot = prepro(trainSet, realPatients, realPivot)
    testData, testPatients, testPivot = prepro(testSet, realPatients, realPivot)
    scoreSum = dict()
    runtimeAll = 0
    generalRecallSum = 0
    generalPrecisionSum = 0
    generalf1ScoreSum = 0
    with tqdm(total=testData.shape[0]) as pbar:
        for index, rows in testData.iterrows():
            # print(rows)
            result, recall, precision, f1score, runtime = predict(rows,trainData, testPivot,trainPivot,testPatients,trainPatients,93,59, medListBelowThres) #nanti coba 70an,60 an atau 150,40
            firstICD10 = json.loads(rows['differentialDiagnosis'])[0]
            #print(firstICD10)
            if str(firstICD10) not in scoreSum:
               scoreSum[firstICD10] = {'Recall': recall, 
                                       'Precision': precision, 
                                       'F1 Score': f1score}
               #print(scoreSum[firstICD10])
               generalRecallSum += recall
               generalPrecisionSum += precision
               generalf1ScoreSum += f1score
               runtimeAll += runtime
               pbar.update(1)
            else:
               scoreSum[firstICD10] = {'Recall': scoreSum[firstICD10]['Recall'] + recall, 
                                       'Precision': scoreSum[firstICD10]['Precision'] + precision, 
                                       'F1 Score': scoreSum[firstICD10]['F1 Score'] + f1score}              
               #print(scoreSum[firstICD10])
               generalRecallSum += recall
               generalPrecisionSum += precision
               generalf1ScoreSum += f1score
               runtimeAll += runtime
               pbar.update(1)
    allScoreSum[i] = scoreSum
    avgRecall = generalRecallSum/len(data)
    avgPresicion = generalPrecisionSum/len(data)
    avgF1Score = generalf1ScoreSum/len(data)
    avgRuntime = runtimeAll/len(data)
    #print(f'Average Recall @K : {generalRecallSum/len(data[i])}')
    #print(f'Average Precision @K : {generalPrecisionSum/len(data[i])}')
    #print(f'Average F1 Score @K : {generalf1ScoreSum/len(data[i])}')
    #print(f'Average Runtime : {runtimeAll/len(data[i])}')
  return allScoreSum, avgRecall, avgPresicion, avgF1Score, avgRuntime

In [ ]:
dataCollection = splitKFold(realPresc, k=10)

In [ ]:
dfgdfgfdgfdgdgddfgcvbcbvc

In [ ]:
allScoreSum, avgRecall, avgPresicion, avgF1Score, avgRuntime =validate(dataCollection, k=10)

  0%|          | 0/9 [00:00<?, ?it/s]

Start of 1-st/th Fold Validation


  0%|          | 0/852 [00:00<?, ?it/s]

Start of 2-st/th Fold Validation


  0%|          | 0/852 [00:00<?, ?it/s]

...

  0%|          | 0/852 [00:00<?, ?it/s]

Start of 8-st/th Fold Validation


  0%|          | 0/852 [00:00<?, ?it/s]

Start of 9-st/th Fold Validation


  0%|          | 0/852 [00:00<?, ?it/s]

In [ ]:
print(f'Average Recall @K : {avgRecall}')
print(f'Average Precision @K : {avgPresicion}')
print(f'Average F1 Score @K : {avgF1Score}')
print(f'Average Runtime : {avgRuntime}')

Average Recall @K : 62.08333333333332
Average Precision @K : 14.711111111111073
Average F1 Score @K : 23.08824508824517
Average Runtime : 138.74079997766785


In [ ]:
print(allScoreSum)

{1: {'A09.0 Other and unspecified gastroenteritis and colitis of infectious origin': {'Recall': 3.4166666666666665, 'Precision': 0.7999999999999999, 'F1 Score': 1.2514152514152515}, 'G44.1 Vascular headache, not elsewhere classified': {'Recall': 1.0, 'Precision': 0.2, 'F1 Score': 0.33333333333333337}, 'K30 Functional dyspepsia': {'Recall': 29.51666666666667, 'Precision': 6.5, 'F1 Score': 10.410323010323008}, 'J02.9 Acute pharyngitis, unspecified': {'Recall': 20.166666666666664, 'Precision': 4.900000000000001, 'F1 Score': 7.815018315018314}, 'K05.6 Periodontal disease, unspecified': {'Recall': 0.5, 'Precision': 0.1, 'F1 Score': 0.16666666666666669}, 'A03.0 Shigellosis due to Shigella dysenteriae': {'Recall': 0.0, 'Precision': 0.0, 'F1 Score': 0}, 'I89.0 Lymphoedema, not elsewhere classified': {'Recall': 1.0, 'Precision': 0.3, 'F1 Score': 0.4615384615384615}, 'I10 Essential (primary) hypertension': {'Recall': 1.0, 'Precision': 0.2, 'F1 Score': 0.33333333333333337}, 'K64.9 Haemorrhoids, u

In [ ]:
allScoreSum

{1: {'A03.0 Shigellosis due to Shigella dysenteriae': {'F1 Score': 0,
   'Precision': 0.0,
   'Recall': 0.0},
  'A09.0 Other and unspecified gastroenteritis and colitis of infectious origin': {'F1 Score': 1.2514152514152515,
   'Precision': 0.7999999999999999,
   'Recall': 3.4166666666666665},
  'A09.9 Gastroenteritis and colitis of unspecified origin': {'F1 Score': 2.432634032634033,
   'Precision': 1.5,
   'Recall': 7.933333333333333},
  'A15.3 Tuberculosis of lung, confirmed by unspecified means': {'F1 Score': 0,
   'Precision': 0.0,
   'Recall': 0.0},
  'A18.2 Tuberculous peripheral lymphadenopathy': {'F1 Score': 0.33333333333333337,
   'Precision': 0.2,
   'Recall': 1.0},
  'A49.2 Haemophilus influenzae infection, unspecified site': {'F1 Score': 1.1585081585081585,
   'Precision': 0.7,
   'Recall': 4.0},
  'A49.9 Bacterial infection, unspecified': {'F1 Score': 1.3901098901098903,
   'Precision': 0.9,
   'Recall': 3.25},
  'A51.9 Early syphilis, unspecified': {'F1 Score': 0,
   'Pr

In [ ]:
#Save File
with open("/content/drive/MyDrive/Internship (Retracing Project)/Data/Progress 5 (KNN)/allScoreSum_Refinement13.json", "w") as outfile:
  json.dump(allScoreSum, outfile)